In [2]:
%pylab inline
#Autoreload so changing py files works
%load_ext autoreload
%autoreload 2
from Util import *
from superpoint import *
#Required for SuperPoint
#from VideoStabilization import *

Populating the interactive namespace from numpy and matplotlib


C:\Users\MasterMatthew\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['show', 'resize', 'flip']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
#Path to input video (0 to use webcam)
inputpath = 0#'data/chair.mp4'
#Set resize to a tuple to resize frames
resize = False#(800, 450)
#Modulus
skip = 1
#Total frames that should be ran
max_frames = 9999
#Show images as they are corrected in a CV frame
show = True
#Enable flipping if images aren't in landscape format
flip = False

#Uses SuperPoint WARNING: May not work properly!
SUPERPOINT=False
if SUPERPOINT:
    extractor = SUPERFeatureExtractor()
else:
    extractor = SIFTFeatureExtractor()
    matcher = BFMatcherHomographyGenerator()

#Reads in frames
def read():
    ret, frame = cap.read()
    if flip and frame.shape[0] < frame.shape[1]:
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 1)
    if resize:
        frame = cv2.resize(frame, resize)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color = frame#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return color, gray

cap = cv2.VideoCapture(inputpath)
correctedFrames = []
count = 0
H = np.array([[1,0,0],[0,1,0],[0,0,1]])

#Read first frame
color1, gray1 = read()
correctedFrames.append(color1)
if SUPERPOINT:
    kps1, desc1, heatmap1 = extractor.extractFeatures(gray1)
else:
    kps1, desc1 = extractor.extractFeatures(gray1)

while cap.isOpened():
    #Read in next frame
    color2, gray2 = read()
    count = count + 1
    #Allow frame skipping
    if count > max_frames: break
    if count % skip == 0:
        #Extract features and find Homography
        if SUPERPOINT:
            kps2, desc2, heatmap2 = extractor.extractFeatures(gray2)
            matches = match(desc1, desc2)
            pair1, pair2 = convertToPoints(kps1, kps2, matches)
            HG, mask = cv2.findHomography(pair2, pair1, cv2.RANSAC, 5.0)
        else:
            kps2, desc2 = extractor.extractFeatures(gray2)
            HG = matcher.findHomography(kps2, desc2, kps1, desc1)
        #Multiply new Homography with previous Homography
        H = H @ HG
        #Warp image by homography
        corrected = cv2.warpPerspective(color2, H, (color2.shape[1], color2.shape[0]))
        correctedFrames.append(corrected)
        kps1 = kps2
        desc1 = desc2
        #Render to CV frame
        if show:
            cv2.imshow('frame', corrected)
    #Press space to end
    if cv2.waitKey(32) == 32:
        break
    
cap.release()